<a href="https://colab.research.google.com/github/savitha91/SpeechToImage/blob/master/NeMo_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tarfile

zipPath ='/content/drive/My Drive/MadeWithML_Project/datasets/MELD/sample/dev.tar.gz'

if zipPath.endswith("tar.gz"):
    tar = tarfile.open(zipPath, "r:gz")
    tar.extractall('/content/VideoFile')
    tar.close()
elif zipPath.endswith("tar"):
    tar = tarfile.open(zipPath, "r:")
    tar.extractall('/content/VideoFile')
    tar.close()



In [ ]:
import os
import pathlib
extractedFilesPath = '/content/VideoFile/dev_splits_complete'
count = 0
for path in pathlib.Path(extractedFilesPath).iterdir():
    if path.is_file():
        count += 1

print("Number of video files available", count)

Number of video files available 1112


In [ ]:
! pip install ffmpeg
! pip install librosa
#!pip install unidecode

  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6084 sha256=1b42e5fad4930182fd005fa72520ae6b3aab2968ae1cdd7d7e26f1e20d36b351
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg


In [ ]:
import sys
import os
import pandas as pd
import json
import codecs
#import unidecode
import librosa
import numpy as np
import pandas as pd
extractedFilesPath = '/content/VideoFile/dev_splits_complete'
csvPath = '/content/drive/My Drive/MadeWithML_Project/datasets/MELD/sample/dev_sent_emo.csv'
manifestJsonPath = '/content/VideoFile/manifestJson.json'

def normalize_str(txt) -> str:
    # TODO: REPLACE WITH YOUR OWN NORMALIZATION LOGIC HERE!!!!   
    valid_chars = (" ", "'", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z")
    new_txt = unidecode.unidecode(txt.lower().strip())
    res_arr = []
    for c in new_txt:
        if c in valid_chars:
            res_arr.append(c)
        else:
            res_arr.append(' ')
    res = ''.join(res_arr).strip()    
    return ' '.join(res.split())

def csv_to_manifest(csv_file, manifest_file):
  manifests = []
  df = pd.read_csv(csv_file, encoding='utf8')
  for index, row in df.iterrows():
    try:
      entry = {} #Dictionary
      wav_folder = '/content/VideoFile/wav'
      os.system('mkdir -p {}'.format(wav_folder))
      mp4_file = "dia"+str(row['Dialogue_ID'])+"_"+"utt"+str(row['Utterance_ID'])+".mp4"
      actual_filename = mp4_file[:-4]
      wav_file_Path = "{0}/{1}".format(wav_folder,actual_filename) + ".wav"
      os.system('ffmpeg -i {}/{} -acodec pcm_s16le -ac 1 -ar 16000 {}/{}.wav'.format(extractedFilesPath,mp4_file,wav_folder, actual_filename))      
      samples,sample_rate= librosa.load(wav_file_Path, sr=16000)
      #duration=librosa.core.get_duration(filename='/content/VideoFile/wav/dia0_utt0.wav')
      audio_duration=np.round(samples.shape[0]/sample_rate , 3)
      entry['audio_filepath'] = wav_file_Path
      entry['duration'] = float(audio_duration)
      entry['text'] = row['Utterance']
      manifests.append(entry)
    except:
      print("SOMETHING WENT WRONG with", wav_folder)

  with codecs.open(manifest_file, 'w', encoding='utf-8') as fout:
    for m in manifests:
      fout.write(json.dumps(m, ensure_ascii=False) + '\n')
  print('Done!')


def main():
  csv_to_manifest(csvPath, manifestJsonPath)
  

if __name__ == "__main__":
    main()

SOMETHING WENT WRONG with /content/VideoFile/wav
Done!


import chardet

import pandas as pd

with open(csvPath, 'rb') as f:
    result = chardet.detect(f.read())  

Check for encoding type when the csv was initially created

----- Convert utf encoded string to ascii-----


b'Oh my God, he\x92s lost it. He\x92s totally lost it.'.decode('windows-1252')

------ Result : Oh my God, he’s lost it. He’s totally lost it.------

-----base64 encoding and decoding a string ----

import base64
Str = "this is string example....wow!!!"
encoded = base64.b64encode(b'this is string example....wow!!!'). 
data = base64.b64decode(encoded)

--- zip a file---



import os
import zipfile

zf = zipfile.ZipFile("wavFiles.zip", "w")
for dirname, subdirs, files in os.walk("mydirectory"):
    zf.write(dirname)
    for filename in files:
        zf.write(os.path.join(dirname, filename))
zf.close()

In [ ]:
!python --version
!pip install deepspeech==0.6.0
# https://colab.research.google.com/github/scgupta/yearn2learn/blob/master/speech/asr/deepspeech06/mozilla_deepspeech_api_notebook.ipynb#scrollTo=XdzZteC7TZDP
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/deepspeech-0.6.0-models.tar.gz
!tar -xvzf deepspeech-0.6.0-models.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   641  100   641    0     0   2823      0 --:--:-- --:--:-- --:--:--  2823
100 1172M  100 1172M    0     0  30.4M      0  0:00:38  0:00:38 --:--:-- 32.0M
deepspeech-0.6.0-models/
deepspeech-0.6.0-models/lm.binary
deepspeech-0.6.0-models/output_graph.pbmm
deepspeech-0.6.0-models/output_graph.pb
deepspeech-0.6.0-models/trie
deepspeech-0.6.0-models/output_graph.tflite


In [ ]:
!deepspeech --model deepspeech-0.6.0-models/output_graph.pb --lm deepspeech-0.7.0-models/lm.binary --trie ./deepspeech-0.6.0-models/trie --audio /content/VideoFile/wav/dia10_utt0.wav

Loading model from file deepspeech-0.6.0-models/output_graph.pb
TensorFlow: v1.14.0-21-ge77504a
DeepSpeech: v0.6.0-0-g6d43e21
2020-07-27 06:04:52.732192: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
Loaded model in 0.128s.
Loading language model from files deepspeech-0.7.0-models/lm.binary ./deepspeech-0.6.0-models/trie
[native_client/ctcdecode/scorer.cpp:77] FATAL: "(access(filename, 4)) == (0)" check failed. Invalid language model path

In [ ]:
import deepspeech
model_file_path = '/content/deepspeech-0.6.0-models/output_graph.pbmm'
beam_width = 500
model = deepspeech.Model(model_file_path, beam_width)
lm_file_path = '/content/deepspeech-0.6.0-models/lm.binary'
trie_file_path = '/content/deepspeech-0.6.0-models/trie'
lm_alpha = 0.75
lm_beta = 1.85
model.enableDecoderWithLM(lm_file_path, trie_file_path, lm_alpha, lm_beta)
import wave
filename = '/content/VideoFile/wav/dia0_utt0.wav'
w = wave.open(filename, 'r')
rate = w.getframerate()
frames = w.getnframes()
buffer = w.readframes(frames)
print(rate)
print(model.sampleRate())
print(str(type(buffer)))
text = model.stt(data16)
print(text)

In [ ]:

#quartzModelPath = '/content/multidataset_quartznet15x5_2.zip'

In [ ]:
 #@title
import os
from os.path import exists, join, basename, splitext
#from IPython.display import YouTubeVideo

#!pip -q install wget youtube-dl wget tensorboardX kaldi-io marshmallow num2words ruamel.yaml soundfile sox torch-stft unidecode
!pip install -q nemo-toolkit==0.9.0 nemo-asr==0.9.0 #--no-deps
!pip install nemo_toolkit[nlp] #BERT pre-trained LM

# apt-get update && apt-get install -y libsndfile1 ffmpeg && pip install Cython
# we need also Apex
if not exists('apex'):
  !git clone -q --depth 1 https://github.com/NVIDIA/apex
  !cd apex && pip install -q --no-cache-dir ./

     |████████████████████████████████| 81kB 5.9MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 317kB 12.4MB/s 
     |████████████████████████████████| 7.4MB 18.2MB/s 
     |████████████████████████████████| 102kB 13.9MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 245kB 51.6MB/s 
     |████████████████████████████████| 112kB 64.0MB/s 
     |████████████████████████████████| 552kB 53.6MB/s 


In [ ]:
#@title
if not exists('quartznet15x5_multidataset'):
  # download the pretrained weights
  !wget -nc -q --show-progress -O quartznet15x5.zip https://api.ngc.nvidia.com/v2/models/nvidia/multidataset_quartznet15x5/versions/1/zip
  !unzip quartznet15x5.zip && mkdir quartznet15x5_multidataset && mv Jasper* quartznet15x5.yaml quartznet15x5_multidataset

quartznet15x5.zip   100%[===================>]  67.86M  19.1MB/s    in 4.7s    
Archive:  quartznet15x5.zip
  inflating: JasperDecoderForCTC-STEP-243800.pt  
  inflating: JasperEncoder-STEP-243800.pt  
  inflating: quartznet15x5.yaml      


In [ ]:

import json
from ruamel.yaml import YAML
import nemo
import nemo_asr

WORK_DIR = "/content/quartznet15x5_multidataset"
MODEL_YAML = "/content/quartznet15x5_multidataset/quartznet15x5.yaml"
CHECKPOINT_ENCODER = "/content/quartznet15x5_multidataset/JasperEncoder-STEP-243800.pt"
CHECKPOINT_DECODER = "/content/quartznet15x5_multidataset/JasperDecoderForCTC-STEP-243800.pt"
# Set this to True to enable beam search decoder
ENABLE_NGRAM = False
# This is only necessary if ENABLE_NGRAM = True. Otherwise, set to empty string
LM_PATH = "<PATH_TO_KENLM_BINARY>"


# Instantiate necessary Neural Modules
# Note that data layer is missing from here



Could not import torchaudio. Some features might not work.


################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [ ]:
def quartzNetMultiDatasetModel_NoLM(manifest, greedy=True):
    from ruamel.yaml import YAML
    yaml = YAML(typ="safe")
    with open(MODEL_YAML) as f:
        jasper_model_definition = yaml.load(f)
    labels = jasper_model_definition['labels']

    neural_factory = nemo.core.NeuralModuleFactory(
    placement=nemo.core.DeviceType.GPU,
    log_dir='/content/tb_logs/',
    backend=nemo.core.Backend.PyTorch,create_tb_writer=True)

    data_preprocessor = nemo_asr.AudioToMelSpectrogramPreprocessor(factory=neural_factory)

    jasper_encoder = nemo_asr.JasperEncoder(
      jasper=jasper_model_definition['JasperEncoder']['jasper'],
      activation=jasper_model_definition['JasperEncoder']['activation'],
      feat_in=jasper_model_definition['AudioToMelSpectrogramPreprocessor']['features'])

    jasper_encoder.restore_from(CHECKPOINT_ENCODER, local_rank=0)


    jasper_decoder = nemo_asr.JasperDecoderForCTC(
        feat_in=1024,
        num_classes=len(labels))
    jasper_decoder.restore_from(CHECKPOINT_DECODER, local_rank=0)

    greedy_decoder = nemo_asr.GreedyCTCDecoder()
    ctc_loss = nemo_asr.CTCLossNM(num_classes=len(labels))

    # Instantiate necessary neural modules
    data_layer = nemo_asr.AudioToTextDataLayer(
        shuffle=False,
        manifest_filepath=manifest,
        labels=labels, batch_size=1)

    # Define inference DAG
    audio_signal, audio_signal_len, transcript, transcript_len = data_layer()

    processed_signal, processed_signal_len = data_preprocessor(
        input_signal=audio_signal,
        length=audio_signal_len)
    print("processed_signal", processed_signal)
    encoded, encoded_len = jasper_encoder(audio_signal=processed_signal,
                                          length=processed_signal_len)
    print("Encoded", encoded)
    log_probs = jasper_decoder(encoder_output=encoded)
    print("Decoder output logprobs",log_probs)
    predictions = greedy_decoder(log_probs=log_probs)
    print("Greedy decoder pred", predictions)
    loss = ctc_loss(
            log_probs=log_probs,
            targets=transcript,
            input_length=encoded_len,
            target_length=transcript_len)
    print("Loss", loss)
    if ENABLE_NGRAM:
        print('Running with beam search')
        beam_predictions = beam_search_with_lm(
            log_probs=log_probs, log_probs_length=encoded_len)
        eval_tensors = [beam_predictions]

    if greedy:
        eval_tensors = [predictions]

    tensors = neural_factory.infer(tensors=eval_tensors)
    if greedy:
        from nemo_asr.helpers import post_process_predictions
        prediction = post_process_predictions(tensors[0], labels)
    else:
        prediction = tensors[0][0][0][0][1]
    return (transcript,prediction)



In [ ]:
transcript,prediction = quartzNetMultiDatasetModel_NoLM(manifestJsonPath)

2020-07-25 11:52:23,528 - INFO - PADDING: 16
2020-07-25 11:52:23,528 - INFO - PADDING: 16
2020-07-25 11:52:23,528 - INFO - PADDING: 16


STFT using torch


2020-07-25 11:52:24,269 - INFO - Dataset loaded with 0.96 hours. Filtered 0.00 hours.
2020-07-25 11:52:24,269 - INFO - Dataset loaded with 0.96 hours. Filtered 0.00 hours.
2020-07-25 11:52:24,269 - INFO - Dataset loaded with 0.96 hours. Filtered 0.00 hours.
2020-07-25 11:52:24,274 - INFO - Evaluating batch 0 out of 1106
2020-07-25 11:52:24,274 - INFO - Evaluating batch 0 out of 1106
2020-07-25 11:52:24,274 - INFO - Evaluating batch 0 out of 1106


processed_signal 0-><class 'nemo.core.neural_types.BatchTag'>:None:None
1-><class 'nemo.core.neural_types.MelSpectrogramSignalTag'>:None:None
2-><class 'nemo.core.neural_types.ProcessedTimeTag'>:None:None
Encoded 0-><class 'nemo.core.neural_types.BatchTag'>:None:None
1-><class 'nemo.core.neural_types.EncodedRepresentationTag'>:None:None
2-><class 'nemo.core.neural_types.ProcessedTimeTag'>:None:None
Decoder output logprobs 0-><class 'nemo.core.neural_types.BatchTag'>:None:None
1-><class 'nemo.core.neural_types.TimeTag'>:None:None
2-><class 'nemo.core.neural_types.ChannelTag'>:None:None
Greedy decoder pred 0-><class 'nemo.core.neural_types.BatchTag'>:None:None
1-><class 'nemo.core.neural_types.TimeTag'>:None:None
Loss non-tensor object


2020-07-25 11:52:30,851 - INFO - Evaluating batch 110 out of 1106
2020-07-25 11:52:30,851 - INFO - Evaluating batch 110 out of 1106
2020-07-25 11:52:30,851 - INFO - Evaluating batch 110 out of 1106
2020-07-25 11:52:37,481 - INFO - Evaluating batch 220 out of 1106
2020-07-25 11:52:37,481 - INFO - Evaluating batch 220 out of 1106
2020-07-25 11:52:37,481 - INFO - Evaluating batch 220 out of 1106
2020-07-25 11:52:44,252 - INFO - Evaluating batch 330 out of 1106
2020-07-25 11:52:44,252 - INFO - Evaluating batch 330 out of 1106
2020-07-25 11:52:44,252 - INFO - Evaluating batch 330 out of 1106
2020-07-25 11:52:50,906 - INFO - Evaluating batch 440 out of 1106
2020-07-25 11:52:50,906 - INFO - Evaluating batch 440 out of 1106
2020-07-25 11:52:50,906 - INFO - Evaluating batch 440 out of 1106
2020-07-25 11:52:57,550 - INFO - Evaluating batch 550 out of 1106
2020-07-25 11:52:57,550 - INFO - Evaluating batch 550 out of 1106
2020-07-25 11:52:57,550 - INFO - Evaluating batch 550 out of 1106
2020-07-25

In [ ]:
prediction[:5]

["bak god he's lost int es to alaska",
 'whatdid',
 'or we could go to the bank close our accounts and cut them off at the swords',
 "you're at ceans",
 "but then we won't be backbutdies"]

In [ ]:
def quartzNetNemoModel_NoLM(manifest, greedy=True):
    from ruamel.yaml import YAML
    yaml = YAML(typ="safe")
    with open(MODEL_YAML) as f:
        jasper_model_definition = yaml.load(f)
    labels = jasper_model_definition['labels']

    neural_factory = nemo.core.NeuralModuleFactory(
    placement=nemo.core.DeviceType.GPU,
    log_dir='/content/tb_logs/',
    backend=nemo.core.Backend.PyTorch,create_tb_writer=True)

    data_preprocessor = nemo_asr.AudioToMelSpectrogramPreprocessor(factory=neural_factory)

    jasper_encoder = nemo_asr.JasperEncoder(
      jasper=jasper_model_definition['JasperEncoder']['jasper'],
      activation=jasper_model_definition['JasperEncoder']['activation'],
      feat_in=jasper_model_definition['AudioToMelSpectrogramPreprocessor']['features'])

    jasper_encoder.restore_from(CHECKPOINT_ENCODER, local_rank=0)


    jasper_decoder = nemo_asr.JasperDecoderForCTC(
        feat_in=1024,
        num_classes=len(labels))
    jasper_decoder.restore_from(CHECKPOINT_DECODER, local_rank=0)

    greedy_decoder = nemo_asr.GreedyCTCDecoder()
    ctc_loss = nemo_asr.CTCLossNM(num_classes=len(labels))

    # Instantiate necessary neural modules
    data_layer = nemo_asr.AudioToTextDataLayer(
        shuffle=False,
        manifest_filepath=manifest,
        labels=labels, batch_size=1)

    # Define inference DAG
    audio_signal, audio_signal_len, transcript, transcript_len = data_layer()

    processed_signal, processed_signal_len = data_preprocessor(
        input_signal=audio_signal,
        length=audio_signal_len)
    print("processed_signal", processed_signal)
    encoded, encoded_len = jasper_encoder(audio_signal=processed_signal,
                                          length=processed_signal_len)
    print("Encoded", encoded)
    log_probs = jasper_decoder(encoder_output=encoded)
    print("Decoder output logprobs",log_probs)
    predictions = greedy_decoder(log_probs=log_probs)
    print("Greedy decoder pred", predictions)
    loss = ctc_loss(
            log_probs=log_probs,
            targets=transcript,
            input_length=encoded_len,
            target_length=transcript_len)
    print("Loss", loss)
    if ENABLE_NGRAM:
        print('Running with beam search')
        beam_predictions = beam_search_with_lm(
            log_probs=log_probs, log_probs_length=encoded_len)
        eval_tensors = [beam_predictions]

    if greedy:
        eval_tensors = [predictions]

    tensors = neural_factory.infer(tensors=eval_tensors)
    if greedy:
        from nemo_asr.helpers import post_process_predictions
        prediction = post_process_predictions(tensors[0], labels)
    else:
        prediction = tensors[0][0][0][0][1]
    return (transcript,prediction)



In [ ]:

Literature survey
Infer on MELD dataset
Accuracy - 60-70%

# TODOs
QuartzNet Multi-dataset without LM 
QuartzNet Multi-dataset with LM (BERT , Khaldi's LM) #https://docs.nvidia.com/deeplearning/nemo/developer_guide/en/stable/
QuartzNet Multi-dataset without LM : .nemo tar file
QuartzNet Nemo model without LM # https://colab.research.google.com/drive/1v_PPJQfNCkAvpc_x-eordoMzz1FaxQGd#scrollTo=jLtlNuMu0LSl
DeepSpeech #https://colab.research.google.com/drive/1bOWBv1uivHdrQUeSPDLyzPsdvThbCXR7#scrollTo=mRTU7BdB389u

For all these find the avergae WER  for the dev dataset--  get text from the transcript Neural Module

!pip install jiwer
wer = jiwer.wer(expected_output, predicted_output)

# Understand what is CTC loss , Quartznet mode architecture
As per papers what are the WER on train and test sets for all the models


Improving using BERT LM
Tried on our instruction

In [ ]:
Speech to text
# infered on  MELD training dataset 
# Use Pretrained LM 
# identify accuracy

#Instruction - infer


In [ ]:
Text to image
# GAN : Nvidia
# Coco dataset

